# Lab 10: MPI

MPI is a standard specification of a message passing interface

<img align="right"  src="images/mpi1.png" alt="Drawing" style="width: 300px;"/>

<div style="text-align: left"> 

    SPMD (Single program multiple data):
    
    - Execution starts in parallel
    
    - MPI implements SPMD
    
    -Static parallelism: #processes doesn't change
    
    
    Fork / Join :
    
    - Execution starts serial
    
    - New processes created at fork
    
    - Used in pthreads
    
    - Dynamic parallelism
    
    
</div>


### MPI - system components

<img  src="images/mpi2.png" alt="Drawing" style="width: 600px;"/>

- Node: A single host on network

- Rank: Process executing the MPI program

### MPI - programmer view

<img  src="images/mpi3.png" alt="Drawing" style="width: 600px;"/>

- Nodes are transparent to the programmer, only ranks matter

- Communicator: Group of ranks that can communicate

- Comm world: Communicator that includes all the ranks

^ Images source: HPML lectures

### Point-to-Point Communication

- send() : Comm.send(self, obj, int dest, int tag=0)


    - obj: object to be sent
    - dest: Rank of destination process
    - Tag: Used to differentiate among messages
    
    
- recv() : Comm.recv(self, buf=None, int source=ANY_SOURCE, int tag=ANY_TAG, Status status=None)

    
    - buf: Optional buffer for the data to be received
    - source: Rank of source process
    - Tag: Used to differentiate among messages
    - status: information about the data received, e.g. rank of source, tag

- comm.Get_rank(): Returns the rank of current process

- comm.Get_size(): Returns the total number of processes

#### Python objects (pickle under the hood)

Use lowercase send() and recv() for python objects

In [ ]:
%%writefile ex1.py

from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    data = {'a': 7, 'b': 3.14}
    comm.send(data, dest=1, tag=11)
elif rank == 1:
    data = comm.recv(source=0, tag=11)
    
    print('Message {} received at rank{}:'.format(data, rank))


In [ ]:
!mpiexec -n 2 python ex1.py

#### Numpy arrays

Use uppercase Send() and Recv()

Buffer argument must be specified as [data, TYPE (MPI.DOUBLE)]

In [ ]:
%%writefile ex2.py

from mpi4py import MPI
import numpy

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

# passing MPI datatypes explicitly
if rank == 0:
    data = numpy.arange(1000, dtype='i')
    comm.Send([data, MPI.INT], dest=1, tag=77)
elif rank == 1:
    data = numpy.empty(1000, dtype='i')
    comm.Recv([data, MPI.INT], source=0, tag=77)
    
    print('Message {} received at rank{}:'.format(data, rank))

# automatic MPI datatype discovery
if rank == 0:
    data = numpy.arange(100, dtype=numpy.float64)
    comm.Send(data, dest=1, tag=13)
elif rank == 1:
    data = numpy.empty(100, dtype=numpy.float64)
    comm.Recv(data, source=0, tag=13)
    
    print('Message {} received at rank{}:'.format(data, rank))

In [ ]:
!mpiexec -n 2 python ex2.py

In [ ]:
%%writefile ex3.py

from mpi4py import MPI

comm = MPI.COMM_WORLD

my_rank = comm.Get_rank()
p = comm.Get_size()

if my_rank != 0:
    message = 'Hello from the other rank {}'.format(my_rank)
    comm.send(message, dest = 0)

else:
    for pid in range(1,p):
        message = comm.recv(source = pid)
        print('Process {} receives message: {}'.format(my_rank, message))

In [ ]:
!mpiexec -n 4 python ex3.py

### Non-blocking communication:

Isend(), Irecv() are non blocking:
    
    Process can continue execution and wait later

In [ ]:
%%writefile ex4.py

from mpi4py import MPI
import time

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    data = {'a': 7, 'b': 3.14}
    req = comm.isend(data, dest=1, tag=11)
    req.wait()
    print('Process {} sent {}'.format(rank, data))
    
elif rank == 1:
    req = comm.irecv(source=0, tag=11)
    # do something
    time.sleep(2)
    
    data = req.wait()
    print('Process {} received {}'.format(rank, data))


In [ ]:
!mpiexec -n 2 python ex4.py

### Performance gain

In [ ]:
%%writefile ex5.py

from mpi4py import MPI
import timeit

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

N = 10000000

def parSum():
    if rank == 0: 
        s = sum(range(N//2))
        comm.send(s,dest=2)
        
    elif rank == 1:
        s = sum(range(N//2+1,N))
        comm.send(s,dest=2)
        
    elif rank == 2:
        s1 = comm.recv(source=0)
        s2 = comm.recv(source=1)
        print (s1+s2)


def serSum():
    s = sum(range(N))

if rank == 0:
    
    tp = timeit.Timer("parSum()","from __main__ import parSum")
    print ('Parallel time: {:.4f} sec'.format(tp.timeit(number=10))) 

    ts = timeit.Timer("serSum()","from __main__ import serSum")
    print ('Serial time: {:.4f} sec'.format(ts.timeit(number=10))) 



In [ ]:
!mpiexec -n 3 python ex5.py